# Reward Shaping

$$ R(s,a,s') = R_0(s,a,s') + \Phi(s) $$

其中，$R_0(s,a,s')$　是原始定义的奖赏函数， $\Phi(s)$　是包含先验知识的势函数（potential function）

## $R_0(s,a,s')$ 的定义

| 阿里搜索 |　京东广告 |
| --- | --- |
| <ul><li>若只有点击，ｐｖ中点击商品的总数目</li><li>若购买，购买的价格</li><li>否则，０</li></ul> | <ul><li>若点击，点击产生的广告收入总和</li><li>若购买，购买的价格</li><li>否则，０</li></ul> |

## 奖赏塑形（引入$\Phi(s)$）的动机

**例子一**： 在使用强化学习求解迷宫问题中，可以定义$\Phi(s)$为状态s所在位置与出口的曼哈顿距离（或者其他距离），使得Ａｇｅｎｔ更快地找到潜在的与出口更近的状态。

**例子二**: 以商品点击为例，考虑不同用户Ａ和Ｂ在类似的状态中发生的商品点击行为。若Ａ点击商品价格较高，Ｂ点击商品的价格较低，那么及时Ａ和Ｂ点击的商品数量相同，我们也可以认为相对于Ａ和Ｂ采用的排序策略带来的影响是不同的。

## $\Phi(s)$的电商场景下的定义

$$\Phi(s)=\sum_i^K \mathcal{ML}(i|\mu_\theta(s))$$

其中，

- K为状态ｓ对应ＰＶ中商品的个数
- i表示第i个商品
- $\mu_\theta(s)$为ａｇｅｎｔ在状态ｓ执行的动作
- $\mathcal{ML}(i|\mu_\theta(s))$表示排序策略为$\mu_\theta(s)$时，对商品的点击或者成交的极大似然估计。

因此，$\Phi(s)$表示在状态ｓ上执行动作$\mu_\theta(s)$时，ＰＶ中所有商品能够被点击或者购买的极大似然概率之和。

## 极大似然函数的定义

令商品ｉ的特征向量（价格、销量等）为$x_i=(x_i^1,x_i^2,\ldots,x_i^m)$，则$x_i^T\mu_\theta(s)$即为商品ｉ在状态ｓ下的最终排序分数。　另一方面，令$y_i\in\{0,1\}$为商品ｉ实际被点击或者成交与否的ｌａｂｅｌ，并假设商品ｉ的实际点击或者成交的概率$p_i$与其排序分数$x_i^T\mu_\theta(s)$满足$\ln\frac{p_i}{1-p_i}=x_i^T\mu_\theta(s)$，则商品i的似然概率为

$$\mathcal{ML} = p_i^{y_i}(1-p_i)^{1-y_i}=(\frac{1}{1+\exp(-x_i^T\mu_\theta(s))})^{y_i}(\frac{1}{1+\exp(x_i^T\mu_\theta(s))})^{1-y_i}$$

为了简化计算，我们对两边取对数，得到对数似然概率：

$$\mathcal{ML}_{\log}(i|\mu_\theta(s)) = y_ix_i^T\mu_\theta(s) -\ln (1+\exp(x_i^T\mu_\theta(s)))$$

将一次请求中所有商品的对数似然概率加起来，则有

$$\Phi(s) = \sum_{i=1}^K \bigg\{ y_ix_i^T\mu_\theta(s) -\ln (1+\exp(x_i^T\mu_\theta(s))) \bigg\}$$

## 奖赏塑形方法，考虑点击和购买之后

- 只有的一次请求中，其对应的ｒｅｗａｒｄｓ为 $\Phi_{\textrm{clk}}(s) = \sum_{i=1}^K \bigg\{ y_i^cx_i^T\mu_\theta(s) -\ln (1+\exp(x_i^T\mu_\theta(s))) \bigg\}$
- 如果一次请求中，有发生购买，则需要考虑商品的价格因素，ｒｅｗａｒｄｓ为 $\Phi_{\textrm{buy}}(s) = \sum_{i=1}^K \bigg\{ y_i^px_i^T\mu_\theta(s) -\ln (1+\exp(x_i^T\mu_\theta(s))) + \ln \textrm{price}_i \bigg\}$